In [5]:
%load_ext autoreload
%autoreload 2
""" run_bdd.py

Run example:
run_bdd.py --USE_PARALLEL False --METRICS Hota --TRACKERS_TO_EVAL qdtrack

Command Line Arguments: Defaults, # Comments
    Eval arguments:
        'USE_PARALLEL': False,
        'NUM_PARALLEL_CORES': 8,
        'BREAK_ON_ERROR': True,
        'PRINT_RESULTS': True,
        'PRINT_ONLY_COMBINED': False,
        'PRINT_CONFIG': True,
        'TIME_PROGRESS': True,
        'OUTPUT_SUMMARY': True,
        'OUTPUT_DETAILED': True,
        'PLOT_CURVES': True,
    Dataset arguments:
            'GT_FOLDER': os.path.join(code_path, 'data/gt/bdd100k/bdd100k_val'),  # Location of GT data
            'TRACKERS_FOLDER': os.path.join(code_path, 'data/trackers/bdd100k/bdd100k_val'),  # Trackers location
            'OUTPUT_FOLDER': None,  # Where to save eval results (if None, same as TRACKERS_FOLDER)
            'TRACKERS_TO_EVAL': None,  # Filenames of trackers to eval (if None, all in folder)
            'CLASSES_TO_EVAL': ['pedestrian', 'rider', 'car', 'bus', 'truck', 'train', 'motorcycle', 'bicycle'],
            # Valid: ['pedestrian', 'rider', 'car', 'bus', 'truck', 'train', 'motorcycle', 'bicycle']
            'SPLIT_TO_EVAL': 'val',  # Valid: 'training', 'val',
            'INPUT_AS_ZIP': False,  # Whether tracker input files are zipped
            'PRINT_CONFIG': True,  # Whether to print current config
            'TRACKER_SUB_FOLDER': 'data',  # Tracker files are in TRACKER_FOLDER/tracker_name/TRACKER_SUB_FOLDER
            'OUTPUT_SUB_FOLDER': '',  # Output files are saved in OUTPUT_FOLDER/tracker_name/OUTPUT_SUB_FOLDER
            'TRACKER_DISPLAY_NAMES': None,  # Names of trackers to display, if None: TRACKERS_TO_EVAL
    Metric arguments:
        'METRICS': ['Hota','Clear', 'ID', 'Count']
"""

import sys
import os
import argparse
from multiprocessing import freeze_support
import numpy as np
import random

sys.path.append('../')
import trackeval  # noqa: E402

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# if __name__ == '__main__':
# Command line interface:
default_eval_config = trackeval.Evaluator.get_default_eval_config()
default_eval_config['PRINT_ONLY_COMBINED'] = True
default_dataset_config = trackeval.datasets.BDD100K.get_default_dataset_config()
default_metrics_config = {'METRICS': ['HOTA', 'CLEAR', 'Identity']}
config = {**default_eval_config, **default_dataset_config, **default_metrics_config}  # Merge default configs
parser = argparse.ArgumentParser()
for setting in config.keys():
    if type(config[setting]) == list or type(config[setting]) == type(None):
        parser.add_argument("--" + setting, nargs='+')
    else:
        parser.add_argument("--" + setting)

args = parser.parse_args(args=[]).__dict__
for setting in args.keys():
    if args[setting] is not None:
        if type(config[setting]) == type(True):
            if args[setting] == 'True':
                x = True
            elif args[setting] == 'False':
                x = False
            else:
                raise Exception('Command line parameter ' + setting + 'must be True or False')
        elif type(config[setting]) == type(1):
            x = int(args[setting])
        elif type(args[setting]) == type(None):
            x = None
        else:
            x = args[setting]
        config[setting] = x
eval_config = {k: v for k, v in config.items() if k in default_eval_config.keys()}
dataset_config = {k: v for k, v in config.items() if k in default_dataset_config.keys()}
metrics_config = {k: v for k, v in config.items() if k in default_metrics_config.keys()}

In [3]:
evaluator = trackeval.Evaluator(eval_config)
dataset_list = [trackeval.datasets.BDD100K(dataset_config)]

metrics_list = []
for metric in [trackeval.metrics.HOTA, trackeval.metrics.CLEAR, trackeval.metrics.Identity]:
    if metric.get_name() in metrics_config['METRICS']:
        metrics_list.append(metric())
    if len(metrics_list) == 0:
        raise Exception('No metrics selected for evaluation')


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /home/guest/dev_repo/MOT_eval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : True                          
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : True                          
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

BDD100K Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : /home/guest/dev_repo/MOT_eval/data/gt/bdd100k/bdd100k_val
TRACKERS_FOLDER      : /home/guest/dev_repo/MOT_eval/data/tracke

In [7]:
# import time
# import traceback
# from multiprocessing.pool import Pool
# from functools import partial
# import os
# from trackeval import utils
# from trackeval.utils import TrackEvalException
# from trackeval import _timing
# from trackeval.metrics import Count

# def evaluate(dataset_list, metrics_list):
#     """Evaluate a set of metrics on a set of datasets"""
#     # config = config
#     metrics_list = metrics_list + [Count()]  # Count metrics are always run
#     metric_names = utils.validate_metrics_list(metrics_list)
#     dataset_names = [dataset.get_name() for dataset in dataset_list]
#     print(dataset_names)
#     output_res = {}
#     output_msg = {}

#     for dataset, dataset_name in zip(dataset_list, dataset_names):
#         # Get dataset info about what to evaluate
#         output_res[dataset_name] = {}
#         print(dataset)
#         output_msg[dataset_name] = {}
#         tracker_list, seq_list, class_list = dataset.get_eval_info()
#         print(tracker_list)
#         print('\nEvaluating %i tracker(s) on %i sequence(s) for %i class(es) on %s dataset using the following '
#                 'metrics: %s\n' % (len(tracker_list), len(seq_list), len(class_list), dataset_name,
#                                     ', '.join(metric_names)))
        
#         # Evaluate each tracker
#         for tracker in tracker_list:
#         # if not config['BREAK_ON_ERROR'] then go to next tracker without breaking
#             try:
#                 # Evaluate each sequence in parallel or in series.
#                 # returns a nested dict (res), indexed like: res[seq][class][metric_name][sub_metric field]
#                 # e.g. res[seq_0001][pedestrian][hota][DetA]
#                 print('\nEvaluating %s\n' % tracker)
#                 time_start = time.time()
#                 if config['USE_PARALLEL']:
#                     with Pool(config['NUM_PARALLEL_CORES']) as pool:
#                         _eval_sequence = partial(eval_sequence, dataset=dataset, tracker=tracker,
#                                                     class_list=class_list, metrics_list=metrics_list,
#                                                     metric_names=metric_names)
#                         results = pool.map(_eval_sequence, seq_list)
#                         res = dict(zip(seq_list, results))

#                 else:
#                     res = {}
#                     for curr_seq in sorted(seq_list):
#                         res[curr_seq] = eval_sequence(curr_seq, dataset, tracker, class_list, metrics_list,
#                                                         metric_names)     
#                         # print(res) 
#                     return res                                                                    
                
#             except Exception as err:
#                 continue

# # @_timing.time
# def eval_sequence(seq, dataset, tracker, class_list, metrics_list, metric_names):
#     """Function for evaluating a single sequence"""

#     raw_data = dataset.get_raw_seq_data(tracker, seq) ##ここにbbox_dfとかが入ってほしい。
#     seq_res = {}
#     # print(class_list)
#     for cls in class_list:
#         seq_res[cls] = {}
#         # print(seq_res)
#         data = dataset.get_preprocessed_seq_data(raw_data, cls)
#         # print(data.keys())
#         for metric, met_name in zip(metrics_list, metric_names):
#             seq_res[cls][met_name] = metric.eval_sequence(data)
#     return seq_res


In [24]:
# evaluate(dataset_list, metrics_list)

['BDD100K']
['qdtrack']

Evaluating 1 tracker(s) on 200 sequence(s) for 8 class(es) on BDD100K dataset using the following metrics: HOTA, CLEAR, Identity, Count


Evaluating qdtrack

{'HOTA': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 'DetA': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 'AssA': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 'DetRe': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 'DetPr': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 'AssRe': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 'AssPr': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 'LocA': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 'RHOTA': arr

KeyboardInterrupt: 

In [10]:
metrics_list

In [8]:
#create dummy bounding box
image_num = 5 #number of images
bbox_offset = 0 #検出用バウンディングボックスのオフセット
IOUThreshold = 0.5 #IOU閾値
ap_only = True #calc_ap関数で、APのみを出力するかどうかのトリガー

bboxes_det = []
bboxes_gt = []
classes = []

for image_name in range(image_num):
    for ClassId in [0 ,32] :
        for num in range(0, 1000, 100):
            imageName = f'{image_name}'
            classId = f'{ClassId}'

            bbox_offset = random.randint(0, 100) #Offset of the bounding box(random) <- ハイパラ
            # print(f'bbox_offset: {bbox_offset}')
            

            num_gt = num
            num_det = num + bbox_offset
            # print(f'image_name: {getImageName} , class_id: {getClassId} , num_gt: {num_gt}, num_det: {num_det}')

            x1 = (1 * num_gt , 1 * num_det)
            y1 = (1 * num_gt , 1 * num_det)
            x2 = (1 * num_gt + 50 , 1 * num_det+ 50)
            y2 = (1 * num_gt + 50 , 1 * num_det+ 50)


            getAbsoluteBoundingBox_det = (x1[1], y1[1], x2[1], y2[1])
            getAbsoluteBoundingBox_gt = (x1[0], y1[0], x2[0], y2[0])
            
            
            getConfidence = random.random()

            gt_info = [imageName, classId, 1, getAbsoluteBoundingBox_gt]
            det_info = [imageName, classId, getConfidence ,getAbsoluteBoundingBox_det]

            if classId not in classes:
                classes.append(classId)

            bboxes_det.append(det_info)
            bboxes_gt.append(gt_info)